In [1]:
import os
import pandas as pd
import json

In [2]:
%run utils/__init__.py

In [3]:
%run metrics/__init__.py

In [4]:
classification = True

In [21]:
base_folder = os.path.join(WORKSPACE_DIR,
                           'classification' if classification else 'report_generation')
results_folder = os.path.join(base_folder, 'results')
runs_folder = os.path.join(base_folder, 'runs')
run_names = os.listdir(results_folder)
_tb_run_names = os.listdir(runs_folder)
run_names = [r for r in run_names if r in _tb_run_names or 'dummy' in r]
run_names = list(run_names)
if 'debug' in run_names: run_names.remove('debug')
run_names

['0709_121201_covid-x_densenet-121_lr1e-06_os',
 '0715_232631_cxr14_densenet-121_lr1e-06_os-max4_aug-0_focal_Pneumonia',
 '0717_140104_covid-kaggle_densenet-121_lr1e-06_e0',
 '0713_165918_covid-x_resnet-50_lr1e-06_os',
 '0717_063232_covid-x_densenet-121_lr1e-06_os-max2_aug-0_precxr14',
 '0704_232547_covid-kaggle_resnet-50_lr1e-06',
 '0716_145915_covid-x_densenet-121_lr1e-06_focal-a=0.9-g=2',
 '0716_171955_covid-x_densenet-121_lr1e-06_aug',
 '0704_062601_covid-kaggle_densenet-121_lr1e-06',
 '0704_015027_covid-kaggle_densenet-121_lr1e-06_os',
 '0714_030324_covid-x_resnet-50_lr1e-06_os-max14',
 '0705_013805_covid-kaggle_resnet-50_lr1e-06_noig_os',
 '0714_232500_cxr14_densenet-121_lr1e-06',
 '0715_232418_cxr14_densenet-121_lr1e-06_os-max4_aug-0_Pneumonia',
 '0717_140241_covid-kaggle_resnet-50_lr1e-06_e0',
 '0704_161918_covid-kaggle_densenet-121_lr1e-06_noig',
 '0716_133211_cxr14_densenet-121_lr1e-06_us_aug-0_Pneumonia',
 '0713_192636_covid-x_densenet-121_lr1e-06_os-max14',
 '0717_120222_co

In [22]:
def load_results(suffix=''):
    filename = 'metrics'
    if suffix:
        filename += f'-{suffix}'

    all_results = []

    for run_name in run_names:
        filepath = os.path.join(results_folder, run_name, f'{filename}.json')

        if not os.path.isfile(filepath):
            print('Not found for: ', run_name)
            continue

        with open(filepath, 'r') as f:
            results = json.load(f)

        all_results.append((run_name, results))

    return all_results

In [23]:
all_results = load_results('') #, 'max-words-27')
len(all_results)

32

In [24]:
def create_results_df(results, key):
    run_names = [run_name for run_name, _ in results]
    
    results_dicts = [d[key] for _, d in results]
    
    df = pd.DataFrame(results_dicts, index=run_names)
    return df

In [33]:
results_df_train = create_results_df(all_results, 'train')
results_df_val = create_results_df(all_results, 'val')
results_df_test = create_results_df(all_results, 'test')

### Report generation

In [17]:
results_df_test = results_df_test.loc[results_df_test.index.str.contains('_lstm_')]

In [61]:
results_df_val

,loss,word_acc,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD
0716_211601_lstm-att_lr0.0001_densenet-121,8.513943,0.068213,0.434808,0.277032,0.194837,0.147231,0.263477,0.348825,0.741187


In [80]:
results_df_test

,loss,word_acc,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD
0716_211601_lstm-att_lr0.0001_densenet-121,7.725230,0.055295,0.401821,0.255879,0.174197,1.245062e-01,0.239101,0.331644,0.481737
0717_085045_dummy-words-200,7.452846,0.031153,0.373533,0.085370,0.015814,6.192783e-07,0.118680,0.211350,0.201866
0717_083015_dummy-sentences-100,39.685739,0.048110,0.323524,0.188075,0.116731,7.227757e-02,0.175152,0.220751,0.192516
0716_234501_h-lstm-att_lr0.0001_densenet-121,5.836308,0.081163,0.278639,0.163611,0.106255,7.249889e-02,0.155251,0.260263,0.120244
0717_083603_dummy-sentences-500,39.702269,0.044475,0.315434,0.182265,0.113446,7.201885e-02,0.170791,0.209300,0.192069
0717_084306_dummy-sentences-50,39.666352,0.050461,0.311519,0.183110,0.114710,7.228451e-02,0.170406,0.223176,0.211758
0717_090748_dummy-words-10,7.426205,0.062627,0.332876,0.114587,0.023408,6.150347e-03,0.119255,0.256932,0.176793
0717_083314_dummy-sentences-200,39.682048,0.045503,0.321895,0.185239,0.113506,6.935814e-02,0.172499,0.214820,0.197815
0717_041434_lstm_lr0.0001_densenet-121,6.577675,0.059194,0.437577,0.295265,0.205521,1.498014e-01,0.272041,0.356053,0.607990
0717_080808_dummy-words-50,7.448333,0.038413,0.409490,0.111303,0.023779,4.728711e-03,0.137325,0.234163,0.271556


In [81]:
print_columns = [c for c in results_df_test.columns if c != 'loss' and 'cm' not in c]
print((results_df_test).to_latex(columns=print_columns,
                               float_format='%.3f',
                              ))

\begin{tabular}{lrrrrrrrr}
\toprule
{} &  word\_acc &  bleu1 &  bleu2 &  bleu3 &  bleu4 &  bleu &  rougeL &  ciderD \\
\midrule
0716\_211601\_lstm-att\_lr0.0001\_densenet-121   &     0.055 &  0.402 &  0.256 &  0.174 &  0.125 & 0.239 &   0.332 &   0.482 \\
0717\_085045\_dummy-words-200                  &     0.031 &  0.374 &  0.085 &  0.016 &  0.000 & 0.119 &   0.211 &   0.202 \\
0717\_083015\_dummy-sentences-100              &     0.048 &  0.324 &  0.188 &  0.117 &  0.072 & 0.175 &   0.221 &   0.193 \\
0716\_234501\_h-lstm-att\_lr0.0001\_densenet-121 &     0.081 &  0.279 &  0.164 &  0.106 &  0.072 & 0.155 &   0.260 &   0.120 \\
0717\_083603\_dummy-sentences-500              &     0.044 &  0.315 &  0.182 &  0.113 &  0.072 & 0.171 &   0.209 &   0.192 \\
0717\_084306\_dummy-sentences-50               &     0.050 &  0.312 &  0.183 &  0.115 &  0.072 & 0.170 &   0.223 &   0.212 \\
0717\_090748\_dummy-words-10                   &     0.063 &  0.333 &  0.115 &  0.023 &  0.006 & 0.119 &   0.257

In [74]:
np.mean([0.369, 0.246, 0.171, 0.115])

0.22525

### Classification

In [31]:
# contains = 'covid-x'
# contains = 'cxr14'
contains = 'e0'
# contains = '0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid'
# contains = '0717_101812_covid-x_densenet-121_lr1e-06_os-max2_aug-covid'
contains = '0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid' # WINNER

contains = '0717_101812_covid-x_densenet-121_lr1e-06_os-max2_aug-covid'

In [34]:
results_df_train = results_df_train.loc[results_df_train.index.str.contains(contains)]
results_df_val = results_df_val.loc[results_df_val.index.str.contains(contains)]
results_df_test = results_df_test.loc[results_df_test.index.str.contains(contains)]

In [111]:
results_df_train.sort_values(['acc', 'recall_covid', 'prec_covid'],
                           ascending=False).dropna(axis=1, how='all')

KeyError: 'recall_covid'

In [91]:
results_df_val.sort_values(['acc', 'recall_covid', 'prec_covid'],
                           ascending=False).dropna(axis=1, how='all')

,loss,acc,prec_covid,prec_pneumonia,prec_normal,recall_covid,recall_pneumonia,recall_normal,spec_covid,spec_pneumonia,spec_normal,cm
0717_063232_covid-x_densenet-121_lr1e-06_os-max2_aug-0_precxr14,1.354517,0.873199,0.928571,0.844891,0.890394,0.553191,0.849541,0.908291,0.998509,0.89917,0.849662,"[[24, 14, 9], [2, 477, 66], [0, 88, 708]]"


In [35]:
results_df_test.sort_values(['acc', 'recall_covid', 'prec_covid'],
                            ascending=False).dropna(axis=1, how='all')

,loss,acc,prec_covid,prec_pneumonia,prec_normal,recall_covid,recall_pneumonia,recall_normal,spec_covid,spec_pneumonia,spec_normal,cm
0717_101812_covid-x_densenet-121_lr1e-06_os-max2_aug-covid,0.145496,0.937302,0.939394,0.911184,0.955275,0.93,0.93266,0.941243,0.995943,0.945178,0.943804,"[[91, 7, 2], [1, 561, 32], [2, 58, 825]]"


In [36]:
print_columns = [
    'acc',
    'prec_covid','prec_pneumonia','prec_normal',
    'recall_covid','recall_pneumonia', 'recall_normal',
    'spec_covid','spec_pneumonia', 'spec_normal'
#     'acc_Pneumonia', 'prec_Pneumonia', 'recall_Pneumonia'
]
# print_columns = [c for c in results_df_test.columns if c != 'loss' and 'cm' not in c]

In [37]:
print((results_df_test*100).to_latex(columns=print_columns,
                               float_format='%.2f',
                              ))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &   acc &  prec\_covid &  prec\_pneumonia &  prec\_normal &  recall\_covid &  recall\_pneumonia &  recall\_normal &  spec\_covid &  spec\_pneumonia &  spec\_normal \\
\midrule
0717\_101812\_covid-x\_densenet-121\_lr1e-06\_os-max... & 93.73 &       93.94 &           91.12 &        95.53 &         93.00 &             93.27 &          94.12 &       99.59 &           94.52 &        94.38 \\
\bottomrule
\end{tabular}



### Report-generation: results at different report lengths

In [32]:
vals_words = [20, 25, 27, 33, 44, None]
vals_sents = [3, 4, 5, 6, None]

In [40]:
max_words = vals_words[0]
suffix = f'max-words-{max_words}' if max_words else ''
all_results = load_results(suffix)
results_df_test = create_results_df(all_results, 'test')
results_df_test

,loss,word_acc,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD
0717_041434_lstm_lr0.0001_densenet-121,5.579509,0.149129,0.478934,0.325275,0.256634,0.218300,0.319786,0.395486,1.354237
0716_211601_lstm-att_lr0.0001_densenet-121,7.382415,0.068293,0.442359,0.282490,0.199632,0.150046,0.268632,0.358223,0.775244
0717_015057_h-lstm_lr0.0001_densenet-121,4.548621,0.162518,0.328184,0.195510,0.126410,0.082401,0.183126,0.334782,0.256378
0716_234501_h-lstm-att_lr0.0001_densenet-121,5.157984,0.132504,0.298223,0.179060,0.115951,0.075574,0.167202,0.315891,0.101786
